In [182]:
# We approached the problem in two ways, and both the codes are present in this Jupyter Notebook.
# We are Group-7 & following are our group members:
# Kashyap Sekhada : 23PGAI0102
# Lakshya Soni : 23PGAI00117
# Hanymol B S : 23PGAI0050
# Solomon Odum: 23PGAI00119

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import json
import imdb
import spacy
from pandas import json_normalize
ia = imdb.Cinemagoer()

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
df = pd.read_json(r"Twitter.json")
tweets = df[['text']].copy()

# ------------CODE 1----------------

## Hosts

In [4]:
hosts = tweets[tweets['text'].str.contains('Host', case = False, na= False)]
host_series = pd.Series(hosts['text'])
host_filter = host_series.str.extractall(r"(?P<name>[A-Z][a-z]* [A-Z][a-z]*)")
result = {}
frequency = host_filter['name'].value_counts()
sorted_hosts = frequency.sort_values(ascending = False)
result['hosts'] = sorted_hosts.iloc[:2].index.values.tolist()
result

{'hosts': ['Amy Poehler', 'Tina Fey']}

## Awards

In [5]:
awards = tweets[tweets['text'].str.contains('Best', case = False, na= False)]
award_series = pd.Series(awards['text'])
award_filter = award_series.str.extractall(r"(?P<award>Best [A-Z][a-z]+ [A-Z*a-z ]+ - [A-Z*a-z ]+)")
award_frequency = award_filter['award'].value_counts()
sorted_awards = award_frequency.sort_values(ascending = False)
award = sorted_awards[:25].copy()
award

Best Actor in a Motion Picture - Drama                               2047
Best Motion Picture - Comedy or Musical                              1737
Best Actress in a Motion Picture - Comedy or Musical                 1646
Best Television Series Actor - Drama                                 1570
Best Motion Picture - Drama                                          1537
Best Supporting Actor in a Television Series - Matt Bomer            1524
Best Animated Feature Film - How To Train Your Dragon                1466
Best Actress in a Motion Picture - Drama                             1457
Best Actor in a Motion Picture - Comedy or Musical                   1285
Best Supporting Actor in a Motion Picture - J                        1246
Best Supporting Actress in a Motion Picture - Patricia Arquette      1144
Best Miniseries or Motion Picture Made for Television - Fargo         994
Best Actor in a TV Movie or Miniseries - Billy Bob Thornton           887
Best Actress in a TV Movie or Miniseri

## Winners

In [9]:
winner = award_series.str.extractall(r'(?P<award>Best [A-Z][a-z]+ [^F][A-Z*a-z ]+) goes to (?P<winner>[A-Z][a-z]+ [A-Z*a-z]+)')
winner_frequency = winner.groupby(['award','winner']).size()
sorted_winner = winner_frequency.sort_values(ascending = False)
sorted_winner[:39]

award                                               winner           
Best Actor in a TV Drama                            Kevin Spacey         707
Best Motion Picture Musical or Comedy               The Grand            362
Best Actor in a Motion Picture Drama                Eddie Redmayne       336
Best Supporting Actress in a Motion Picture         Patricia Arquette    194
Best Actress in a TV Comedy or Musical              Gina Rodriguez       180
Best Actor in a Motion Picture Comedy               Michael Keaton       175
Best Actor in a TV Comedy                           Jeffrey Tambor       169
Best Original Song in a Motion Picture              John Legend          166
Best Actor in a TV Drama Series                     Kevin Spacey         166
Best Dressed Men at the Golden Globes               Matt Bomer           156
Best Actress in a Motion Picture Drama              Julianne Moore       146
Best Dressed Couple                                 Diane Kruger         137
Best A

In [12]:
winner_film = award_series.str.extractall(r'(?P<award>Best [A-Z][a-z]+ [FfPp][A-Z*a-z ]+) goes to (?P<winner>[A-Z][a-z]+ [A-Z*a-z ]+)')    
film_frequency = winner_film.groupby(['award','winner']).size()
sorted_film = film_frequency.sort_values(ascending = False)
sorted_film

award                                      winner                                      
Best Motion Picture Musical or Comedy      The Grand Budapest Hotel                        360
Best Animated Feature Film                 How To Train Your Dragon                         82
                                           How to Train Your Dragon                         79
Best Nomination Face of the evening        Louis CK                                         15
Best Animated Feature goes                 How To Train Your Dragon                         11
Best Animated Film                         How To Train Your Dragon                          9
Best Animated Feature                      How To Train Your Dragon                          7
Best Animated Film                         How to Train Your Dragon                          4
Best Motion Picture Comedy or Musical      Wes Anderson                                      3
Best Motion Picture Musical or Comedy      The Grand Buda

# ------------CODE 2----------------

In [13]:
t = open("Twitter.json", encoding = 'utf-8')
data = json.load(t)

# Hosts

In [14]:
'''
This function verifies if the given name is name of an actor.
'''
def check_if_actor_exists(original_name):
    original_name = original_name.lower().strip()
    search_name = original_name.replace(" ", "+")
    result = requests.get(f"https://www.imdb.com/search/name/?name={search_name}")
    soup = BeautifulSoup(result.text)
    all_models = soup.find_all("div", {"class": "lister-item mode-detail"})
    if len(all_models) > 0:
        for single_model in all_models:
            name = single_model.find_all("a")[1].text.lower().strip()
            if name == original_name:
                return True
    return False

In [15]:
'''
This funtion is used to get the hosts of the ceremony
'''
def get_host():
    result = {}
    list_found = []
    for i in data:
        found = re.search('(H|h)ost', i["text"])
        if found:
            doubt = re.search('shoud have|could have|hope|Best of luck|drea(m|ms)',i['text'])
            if not doubt:
                patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
                matches = patt.findall(i["text"])
                for match in matches:
                    if match in result:
                        result[match] += 1
                    else:
                        result[match] = 1
    sorted_hosts = sorted(result.items(), key=lambda x:x[1], reverse=True)
    converted_dict = dict(sorted_hosts)
    hosts = list(converted_dict.keys())[:2]
    return hosts
hosts = get_host()
hosts

['Amy Poehler', 'Tina Fey']

# Awards Category

In [16]:
def get_award():
    result = {}
    for i in data:
        found = re.search('(B|b)est', i["text"])
        if found:
            patt = re.compile(r'Best [A-Z][a-z]+ [A-Za-z ]+ goes to [A-Za-z0-9 ]+')
            matches = patt.findall(i["text"])
            for match in matches:
                cat = match.strip()
                if cat in result:
                    result[cat] += 1
                else:
                    result[cat] = 1
    category_dict = sorted(result.items(), key=lambda x:x[1], reverse=True) # sorts the names according to the frequency
    category_dict = dict(category_dict)
    categories = list(category_dict.keys())
    return categories
awards = get_award()

In [17]:
'''
This function filters out all the categories containing the keywords "Actors" and "Actresses"
'''
def get_person_categories():
    person_category_genre = []
    lst = ['Actress', 'Actor' ]
    for award in awards:
        category = award.split('goes to')[0]
        for i in lst:
            if i in category.split():
                person_category_genre.append(category)
    person_category_genre = list(person_category_genre)
    return person_category_genre[:20]
person_category_genre = get_person_categories()

In [18]:
'''
This function fetches all the remaining categories
'''
def get_movie_category():
    movie_category_genre = []
    lst = ['Actress', 'Actor' ]
    for category in awards:
        # category = award.split('goes to')[0]
        cat_split = category.split()
        if 'Actress' not in cat_split and 'Actor' not in cat_split:
            movie_category_genre.append(category)
    return movie_category_genre[:10]
movie_category_genre = get_movie_category()

# Winners

In [21]:
'''
This function fetches the winners along with the categories containing the keywords "Actors" and "Actresses"
'''
def get_winners():
    win = dict()
    for category in person_category_genre:
        result = {}
        for i in data:
            found = re.search(category, i["text"])
            if found:
                patt1 = re.compile(r'[A-Z][a-z]* [A-Z][a-z]*')
                matches = patt1.findall(i["text"])
                for match in matches:
                    for i in match.split():
                        if i not in category.split():
                            if match.strip() in result:
                                result[match.strip()] += 1
                            else:
                                result[match.strip()] = 1
        sorted_winner = sorted(result.items(), key=lambda x:x[1], reverse=True)
        for winner in sorted_winner:
            win[category] = sorted_winner[:1][0][0]
    return win
winners = get_winners()
winners

{'Best Actor in a TV Drama ': 'Kevin Spacey',
 'Best Actor in a Motion Picture Drama ': 'Eddie Redmayne',
 'Best Actor in a Motion Picture Comedy ': 'Michael Keaton',
 'Best Actor in a TV Comedy ': 'Jeffrey Tambor',
 'Best Actor in a TV Drama Series ': 'Kevin Spacey',
 'Best Actress in a TV Comedy or Musical ': 'Gina Rodriguez',
 'Best Actress in a Motion Picture Drama ': 'Julianne Moore',
 'Best Actress in a Motion Picture Musical or Comedy ': 'Amy Adams',
 'Best Supporting Actress ': 'Patricia Arquette',
 'Best Actor in a Miniseries or TV Movie ': 'Billy Bob',
 'Best Supporting Actor in a Motion Picture ': 'K Simmons',
 'Best Actor in a Drama ': 'Eddie Redmayne',
 'Best Supporting Actress in a Motion Picture ': 'Patricia Arquette',
 'Best Actress in a TV Drama ': 'Ruth Wilson',
 'Best Actress in a TV Movie or Miniseries ': 'Maggie Gyllenhaal',
 'Best Actor in a TV Movie or Miniseries ': 'Billy Bob',
 'Best Actress in Comedy ': 'Amy Adams',
 'Best Supporting Actor in a TV Series ': 'M

In [22]:
'''
Gets the records of all the winners of the remaining categories
'''
def get_winners_rest():
    result = []
    win = dict()
    role = ['actor', 'actress']
    for category in movie_category_genre:
        winner = category.split("goes to")
        if winner not in result:
            result.append(winner)
    return result
winners_rest = get_winners_rest()
winners_rest

[['Best Motion Picture Musical or Comedy ', ' The Grand Budapest Hotel'],
 ['Best Original Song in a Motion Picture ', ' John Legend and Common for'],
 ['Best Dressed Men at the Golden Globes ', ' Matt Bomer'],
 ['Best Dressed Couple ', ' Diane Kruger'],
 ['Best Animated Feature ', ' HOW TO TRAIN YOUR DRAGON 2'],
 ['Best Animated Feature Film ', ' How To Train Your Dragon 2'],
 ['Best Animated Feature Film ', ' How to Train Your Dragon 2'],
 ['Best Foreign Film ', ' LEVIATHAN'],
 ['Best Original Score ', ' THEORY OF EVERYTHING'],
 ['Best Foreign Language Film ', ' Leviathan']]

## Nominee

In [23]:
'''
This function fetches the list of all the nominees
Since this functions uses imdb website to verify the names of actors, it might take time to run the function
'''
def get_nominee():
    result = {}
    list_found = []
    for i in data:
        found = re.search('[Nn]omin[a-z]+', i["text"])
        if found:
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(i["text"])
            for match in matches:
                if match in result:
                    result[match] += 1
                else:
                    result[match] = 1
    sorted_nominee = sorted(result.items(), key=lambda x:x[1], reverse=True)
    sorted_nominee = dict(sorted_nominee)
    for nominee in list(sorted_nominee.keys())[:200]:
        if check_if_actor_exists(nominee): # searches on the imdb website if the actor exists
                list_found.append(nominee)
    return list_found
nominee = get_nominee()
nominee

['Kevin Spacey',
 'Calvin Klein',
 'George Clooney',
 'Bill Murray',
 'Amy Adams',
 'One Direction',
 'Conchita Wurst',
 'Meryl Streep',
 'Ava Duvernay',
 'Rosamund Pike',
 'Matt Lauer',
 'Julianne Moore',
 'The Normal',
 'Gina Rodriguez',
 'Amy Poehler',
 'Susan Lucci',
 'Jennifer Aniston',
 'Keira Knightley',
 'Christoph Waltz',
 'Eddie Redmayne',
 'Benedict Cumberbatch',
 'Kerry Washington',
 'Lois Lane',
 'Julianna Margulies',
 'Jake Gyllenhaal',
 'Don Cheadle',
 'Black List',
 'Amal Clooney',
 'Vera Wang',
 'Gone Girl',
 'Felicity Jones',
 'Big Eyes',
 'James Spader',
 'Tatiana Maslany',
 'Bill Hader',
 'Channing Tatum',
 'Robin Wright',
 'Clive Owen',
 'Maggie Gyllenhaal',
 'Jessica Lange',
 'Viola Davis',
 'Jeffrey Tambor',
 'Emma Stone',
 'Emily Blunt',
 'Jessica Chastain',
 'The Stars',
 'Naomi Watts',
 'Michael Keaton',
 'Matt Bomer',
 'Kristen Wiig',
 'Yemi Alade',
 'Jeremy Renner',
 'Michael Fassbender',
 'Edie Falco',
 'Jane Fonda',
 'Taylor Schilling',
 'Kate Hudson',
 'D

## Presenters

In [24]:
'''
This function fetches the list of all the presenters
Since this functions uses imdb website to verify the names of actors, it might take time to run the function
'''
def get_presenters():
    result = {}
    varified_presenters = []
    for i in data:
        found = re.search('(P|p)resent(ing|ed)', i["text"])
        if found:        
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(i["text"])
            for match in matches:
                if match in result:
                    result[match] += 1
                else:
                    result[match] = 1

    sorted_hosts = sorted(result.items(), key=lambda x:x[1], reverse=True)
    presenters_dict = dict(sorted_hosts)
    d = dict((k, v) for k, v in presenters_dict.items() if v >= 10) #taking presenters name who are repeated more than 10 times.
    for i in d.keys():
        if check_if_actor_exists(i) == True:   #verifing the person name with IMDB
            varified_presenters.append(i)
    return varified_presenters
presenters = get_presenters()
presenters

['Salma Hayek',
 'Jamie Dornan',
 'Dakota Johnson',
 'Naomi Watts',
 'Meryl Streep',
 'Katie Holmes',
 'Jennifer Lopez',
 'Benedict Cumberbatch',
 'Jennifer Aniston',
 'John Legend',
 'Amal Clooney',
 'Bill Hader',
 'Ricky Gervais',
 'Jared Leto',
 'Chris Pratt',
 'Anna Faris',
 'Jeremy Renner',
 'George Clooney',
 'Eddie Redmayne',
 'Kevin Hart',
 'Kristen Wiig',
 'Gina Rodriguez',
 'Joshua Jackson',
 'Julianna Margulies',
 'Amy Adams',
 'Adam Levine',
 'Mike Ruffin',
 'Paul Rudd',
 'Don Cheadle',
 'Walter White',
 'Olivia Pope',
 'Vince Vaughn',
 'Jack Black',
 'Ryan Seacrest',
 'Kerry Washington',
 'Richard Linklater',
 'Harrison Ford',
 'John Nendick',
 'Karen Angel',
 'Andy Sale',
 'Channing Tatum',
 'Katherine Heigl',
 'Kate Hudson',
 'Bryan Cranston',
 'Robert Downey',
 'Katie Cassidy',
 'Lily Tomlin',
 'Jeffrey Tambor',
 'Michael Keaton',
 'Wes Anderson']

## Favorites

In [30]:
# Function to get the favorites
def get_favorites():
    favorites = []
    result = {}
    list_found = []
    for i in data:
        found = re.search('shoud have|could have|hope|excited|my favourite|favourite', i["text"])
        if found:
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(i["text"])
            for match in matches:
                if match in result:
                    result[match] += 1
                else:
                    result[match] = 1
    sorted_hosts = sorted(result.items(), key=lambda x:x[1], reverse=True)
    converted_dict = dict(sorted_hosts)         
    for keys,values in list(converted_dict.items())[:15]:
        if check_if_actor_exists(keys):
            favorites.append(keys)
    return favorites
favorites = get_favorites()
favorites

['Patricia Arquette',
 'Jessica Lange',
 'Tina Fey',
 'George Clooney',
 'Eddie Redmayne',
 'Amal Clooney',
 'Amy Poehler',
 'Meryl Streep',
 'Dakota Johnson',
 'Gina Rodriguez',
 'Diane Kruger']

In [31]:
# Following are the records of all the hosts, presenters, nominee and winners
print('Following are the hosts of the award ceremony\n',hosts)
print('\n\nFollowing is a list of all the presenters of different categories:\n', presenters)
print('\n\nFollowing is a list of all the nominee of different categories:\n', nominee)
print('\n\nFollowing is a record of all the winners of different categories:\n', winners, winners_rest)
print('\n\nFollowing is a list of all the favorites:\n', favorites)

Following are the hosts of the award ceremony
 ['Amy Poehler', 'Tina Fey']


Following is a list of all the presenters of different categories:
 ['Salma Hayek', 'Jamie Dornan', 'Dakota Johnson', 'Naomi Watts', 'Meryl Streep', 'Katie Holmes', 'Jennifer Lopez', 'Benedict Cumberbatch', 'Jennifer Aniston', 'John Legend', 'Amal Clooney', 'Bill Hader', 'Ricky Gervais', 'Jared Leto', 'Chris Pratt', 'Anna Faris', 'Jeremy Renner', 'George Clooney', 'Eddie Redmayne', 'Kevin Hart', 'Kristen Wiig', 'Gina Rodriguez', 'Joshua Jackson', 'Julianna Margulies', 'Amy Adams', 'Adam Levine', 'Mike Ruffin', 'Paul Rudd', 'Don Cheadle', 'Walter White', 'Olivia Pope', 'Vince Vaughn', 'Jack Black', 'Ryan Seacrest', 'Kerry Washington', 'Richard Linklater', 'Harrison Ford', 'John Nendick', 'Karen Angel', 'Andy Sale', 'Channing Tatum', 'Katherine Heigl', 'Kate Hudson', 'Bryan Cranston', 'Robert Downey', 'Katie Cassidy', 'Lily Tomlin', 'Jeffrey Tambor', 'Michael Keaton', 'Wes Anderson']


Following is a list of all